In [3]:
import sys

sys.path.extend(['.', '..'])

import platform
import os
import numpy as np
import tensorflow as tf
import keras
from keras import layers

gpu_token = tf.test.is_gpu_available('GPU')

if platform.machine() == 'x86_64':
    if gpu_token:
        os.chdir('/home/ucdasec/Faulthunter-RNN-approach/manual_dataset')
    else:
        os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach/manual_dataset")
elif platform.machine() == 'arm64':
    os.chdir("/Users/saksh.menon/Documents/GitHub/C-RNN-approach/manual_dataset")

from pycparser import c_parser, c_generator
import pycparser

text=[]
with open('saksh_branch_simple_insecure.c') as obj:
    temp = obj.read()
    obj.seek(0)
    while obj.tell()!=len(temp):
        try:
            line = obj.readline()
            if line.__contains__("#include"):
                pass
            else:
                text.append(line)
        except EOFError:
            break

text = "".join(text)
print(text)



int main(void)
{
    int result = 1;
    if (result > 1)
    {
        if (result < 3) {
            printf("Executing critical code...\n");
        }
    }
    else
    {
        printf("Exiting out...\n");
        return 1;
    }

    return 0;
}


2024-03-27 13:09:35.602765: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-27 13:09:35.602831: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
parser = c_parser.CParser()
generator = c_generator.CGenerator()
ast = parser.parse(text, filename='??.c')
branch_insecurities = {pycparser.c_ast.If: {'key': 'cond', 'branches': ('iftrue', 'iffalse')}} 
                    #    pycparser.c_ast.Switch: (),
                    #    pycparser.c_ast.For: ()}

HAMMING_WEIGHT = 4

In [14]:
parent_ast = ast.children()[0][1].body

NODE_LIST = []

def walk(parent_ast, flag):
    if parent_ast.children():
        sub_ast = parent_ast.children()
        for node in sub_ast:
            if type(node[1]) in list(branch_insecurities.keys()):
                node_attrs = list((branch_insecurities[type(node[1])]['key'],) + branch_insecurities[type(node[1])]['branches'])
                node_attr = {i: False for i in node_attrs}
                for sub_node in node[1].children():
                    node_attr[sub_node[0]] = sub_node

                print(type(node[1]), end = " : ")
                print(node_attr[branch_insecurities[type(node[1])]['key']])
                NODE_LIST.append(node_attr[branch_insecurities[type(node[1])]['key']])
                for sub_node in branch_insecurities[type(node[1])]['branches']:
                    print(sub_node, " : {}".format(type(node_attr[sub_node][1].block_items[0]) if node_attr[sub_node] else node_attr[sub_node]))
                    if  node_attr[sub_node]:
                        walk(node_attr[sub_node][1], 1)
            # elif(flag):
            else:
                print(type(node[1]))
                NODE_LIST.append(type(node[1]))
        
          
walk(parent_ast, 0)



<class 'pycparser.c_ast.Decl'>
<class 'pycparser.c_ast.If'> : ('cond', BinaryOp(op='>',
         left=ID(name='result'
                 ),
         right=Constant(type='int',
                        value='1'
                        )
         ))
iftrue  : <class 'pycparser.c_ast.If'>
<class 'pycparser.c_ast.If'> : ('cond', BinaryOp(op='<',
         left=ID(name='result'
                 ),
         right=Constant(type='int',
                        value='3'
                        )
         ))
iftrue  : <class 'pycparser.c_ast.FuncCall'>
<class 'pycparser.c_ast.FuncCall'>
iffalse  : False
iffalse  : <class 'pycparser.c_ast.FuncCall'>
<class 'pycparser.c_ast.FuncCall'>
<class 'pycparser.c_ast.Return'>
<class 'pycparser.c_ast.Return'>


In [27]:
ast.children()[0][1].body.children()[0][1]

Decl(name='result',
     quals=[
           ],
     align=[
           ],
     storage=[
             ],
     funcspec=[
              ],
     type=TypeDecl(declname='result',
                   quals=[
                         ],
                   align=None,
                   type=IdentifierType(names=['int'
                                             ]
                                       )
                   ),
     init=Constant(type='int',
                   value='1'
                   ),
     bitsize=None
     )

In [23]:
((NODE_LIST)[2][1].right.value) = 'LOW_WEIGHT'

In [24]:
NODE_LIST

[pycparser.c_ast.Decl,
 ('cond', BinaryOp(op='>',
           left=ID(name='result'
                   ),
           right=Constant(type='int',
                          value='LOW_WEIGHT'
                          )
           )),
 ('cond',
  BinaryOp(op='<',
           left=ID(name='result'
                   ),
           right=Constant(type='int',
                          value='LOW_WEIGHT'
                          )
           )),
 pycparser.c_ast.FuncCall,
 pycparser.c_ast.FuncCall,
 pycparser.c_ast.Return,
 pycparser.c_ast.Return]

In [44]:
dir(pycparser.c_ast)

['Alignas',
 'ArrayDecl',
 'ArrayRef',
 'Assignment',
 'BinaryOp',
 'Break',
 'Case',
 'Cast',
 'Compound',
 'CompoundLiteral',
 'Constant',
 'Continue',
 'Decl',
 'DeclList',
 'Default',
 'DoWhile',
 'EllipsisParam',
 'EmptyStatement',
 'Enum',
 'Enumerator',
 'EnumeratorList',
 'ExprList',
 'FileAST',
 'For',
 'FuncCall',
 'FuncDecl',
 'FuncDef',
 'Goto',
 'ID',
 'IdentifierType',
 'If',
 'InitList',
 'Label',
 'NamedInitializer',
 'Node',
 'NodeVisitor',
 'ParamList',
 'Pragma',
 'PtrDecl',
 'Return',
 'StaticAssert',
 'Struct',
 'StructRef',
 'Switch',
 'TernaryOp',
 'TypeDecl',
 'Typedef',
 'Typename',
 'UnaryOp',
 'Union',
 'While',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_repr',
 'sys']